In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
import sqlalchemy
import time
from datetime import date,datetime,timedelta
def save_monthly_weather(year, month, the_tag, table_name):
    # table = BeautifulSoup(table_text,"html.parser")  
    try:
        data=[]
        for sub_table in the_tag.findAll('table',attrs={'aria-labelledby' :"Days data"}):
            sub_data=[]
            for tr in sub_table.findAll('tr', attrs={'class': ['ng-star-inserted']}):
                tds = tr.findAll('td', attrs={'class': ['ng-star-inserted']})
                tds = [ele.text.strip() for ele in tds]
                sub_data.append(tds)
            data.append(sub_data)
            
        ## convert 3 dim raw data to 2 dim data
        df_raw = pd.DataFrame(data)
        dim2_data =[]
        for series_name, series in df_raw.items():
            row = []
            for srow in series:
                for e in srow:
                    row.append(e)
            dim2_data.append(row)
        
        # hard coding header
        new_header =['DATE','TEMPERATURE_MAX','TEMPERATURE_AVG','TEMPERATURE_MIN','DEW_POINT_MAX','DEW_POINT_AVG','DEW_POINT_MIN',
                     'HUMIDITY_MAX','HUMIDITY_AVG','HUMIDITY_MIN','WIND_SPEED_MAX','WIND_SPEED_AVG','WIND_SPEED_MIN',
                     'PRESSURE_MAX','PRESSURE_AVG','PRESSURE_MIN','PRECIPITATION']  
        dim2_data[0] = new_header
        np_data = np.array(dim2_data)
        df = pd.DataFrame(data=np_data[1:,0:],columns=np_data[0,0:])
        
        # df['DATE'] = year +'-' +month +'-'+df['DATE']
        df['DATE'] = df['DATE'].apply( lambda v : datetime.strptime(year+'-'+month+'-'+v,"%Y-%m-%d" ))
        # print(df['DATE'])
        engine = sqlalchemy.create_engine('sqlite:///D:\\sqlite\\data\\finance.db')
        df.to_sql(table_name,engine,if_exists='append') 
        print("monthly weather saved for year {}, month {}",year,month)
    except Exception as e:
        print("monthly weather save failed: year {}, month {}",year,month)
        print(e)

def get_monthly_weather(year, month,baseurl):
    try:
        url = baseurl+year+'-'+month
        # url = 'https://www.wunderground.com/history/monthly/gb/london/EGLC/date/'+year+'-'+month
        print(url)
        driver = webdriver.Chrome()
        driver.get(url)
        
        html = driver.page_source
        soup = BeautifulSoup(html)
        table = soup.find(lambda tag: tag.name=='table' and tag.has_attr('aria-labelledby') and tag['aria-labelledby']=='History days')    
        # print(type(table))
        # print(table['class'])
        # print(table)
        return table
    except Exception as e:
        print("unable get weather data: year {},month {}".format(year,month))
        print(e)
        
def get_and_save(year,month,baseurl, table_name):
    month_data = get_monthly_weather(year,month,baseurl)
    save_monthly_weather(year,month,month_data,table_name)
    

    

In [2]:
start_date = date.fromisoformat("2024-01-02")
end_date = date.fromisoformat("2024-02-01")
# base_url ='https://www.wunderground.com/history/monthly/gb/london/EGLC/date/'
base_url ='https://www.wunderground.com/history/monthly/us/ny/new-york-city/KLGA/date/'
table_name = 'NY_WEATHER'
# table_name = 'LDN_WEATHER'

while start_date < end_date:
    year = str(start_date.year)
    month = str(start_date.month)
    the_data = get_monthly_weather(year,month,base_url)
    save_monthly_weather(year,month,the_data,table_name)
    start_date = start_date + timedelta(days=31)
    # time.sleep(17)

https://www.wunderground.com/history/monthly/us/ny/new-york-city/KLGA/date/2024-1
monthly weather saved for year {}, month {} 2024 1


In [3]:
base_url ='https://www.wunderground.com/history/monthly/gb/london/EGLC/date/'
table_name = 'LDN_WEATHER'
get_and_save('2023','1',base_url,table_name)
    

https://www.wunderground.com/history/monthly/gb/london/EGLC/date/2023-1
monthly weather saved for year {}, month {} 2023 1


In [18]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import sqlalchemy
from datetime import date,datetime,timedelta
text ="""
<table _ngcontent-app-root-c189="" aria-labelledby="History days" class="days ng-star-inserted"><thead _ngcontent-app-root-c189=""><tr _ngcontent-app-root-c189=""><td _ngcontent-app-root-c189="" class="ng-star-inserted">Time</td><td _ngcontent-app-root-c189="" class="ng-star-inserted">Temperature (°F)</td><td _ngcontent-app-root-c189="" class="ng-star-inserted">Dew Point (°F)</td><td _ngcontent-app-root-c189="" class="ng-star-inserted">Humidity (%)</td><td _ngcontent-app-root-c189="" class="ng-star-inserted">Wind Speed (mph)</td><td _ngcontent-app-root-c189="" class="ng-star-inserted">Pressure (in)</td><td _ngcontent-app-root-c189="" class="ng-star-inserted">Precipitation (in)</td><!-- --></tr></thead><tbody _ngcontent-app-root-c189=""><tr _ngcontent-app-root-c189=""><td _ngcontent-app-root-c189="" class="ng-star-inserted"><table _ngcontent-app-root-c189="" aria-labelledby="Days data"><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> Nov </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 1 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 2 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 3 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 4 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 5 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 6 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 7 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 8 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 9 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 10 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 11 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 12 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 13 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 14 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 15 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 16 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 17 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 18 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 19 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 20 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 21 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 22 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 23 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 24 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 25 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 26 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 27 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 28 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30 </td><!-- --></tr><!-- --></table></td><td _ngcontent-app-root-c189="" class="ng-star-inserted"><table _ngcontent-app-root-c189="" aria-labelledby="Days data"><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> Max </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> Avg </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> Min </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 50 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 45.5 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 41 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 50 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 44.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 39 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 56 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 49.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 42 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 61 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 54.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 48 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 64 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 57.5 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 51 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 56 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 51.0 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 45 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 66 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 59.7 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 55 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 54 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 47.9 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 44 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 58 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 51.6 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 44 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 54 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 51.6 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 48 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 55 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 49.1 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 43 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 45 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 41.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 38 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 48 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 43.7 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 37 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 53 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 48.6 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 44 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 53 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 47.8 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 42 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 65 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 55.2 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 47 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 66 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 58.6 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 50 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 61 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 52.5 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 43 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 56 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 48.8 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 43 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 45 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 41.6 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 39 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 48 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 42.8 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 35 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 54 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 52.0 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 49 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 53 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 49.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 46 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 50 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 45.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 34 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 41 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 36.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 32 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 55 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 45.8 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 37 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 54 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 47.5 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 38 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 40 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 36.1 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 34 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 37 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 33.6 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 28 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 51 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 44.2 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 36 </td><!-- --></tr><!-- --></table></td><td _ngcontent-app-root-c189="" class="ng-star-inserted"><table _ngcontent-app-root-c189="" aria-labelledby="Days data"><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> Max </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> Avg </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> Min </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 37 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.9 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 25 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 28 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 25.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 21 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 36 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.7 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 27 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 39 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 35.8 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 33 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 42 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 38.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 36 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 47 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 37.1 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 50 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 47.1 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 38 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 36 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 25.9 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 18 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 46 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 38.7 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 25 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 37 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 31.9 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 26 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 37 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 31.8 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 27 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 27 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 24.6 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 22 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 32 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 27.0 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 21 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 34 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 31.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 35 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 32.2 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 46 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 38.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 31 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 52 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 47.7 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 38 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 54 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 38.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 23 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 31 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 27.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 23 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 25 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 19.2 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 10 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 44 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 31.9 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 20 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 52 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 46.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 42 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 43 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 34.2 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 33 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 25.5 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 14 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 22 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 18.0 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 15 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 52 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 36.0 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 22 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 51 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 35.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 18 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 20 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 15.0 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 12 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 15 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 12.2 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 10 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 21.6 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 13 </td><!-- --></tr><!-- --></table></td><td _ngcontent-app-root-c189="" class="ng-star-inserted"><table _ngcontent-app-root-c189="" aria-labelledby="Days data"><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> Max </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> Avg </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> Min </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 76 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 57.9 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 38 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 65 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 48.9 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 32 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 62 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 49.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 37 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 61 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 50.2 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 36 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 66 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 49.5 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 37 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 75 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 59.6 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 48 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 80 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 63.8 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 50 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 56 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 43.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 72 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 61.7 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 47 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 66 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 48.0 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 34 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 66 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 52.0 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 40 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 60 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 51.7 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 43 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 62 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 52.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 42 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 62 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 51.7 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 41 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 65 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 55.2 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 43 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 69 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 55.0 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 28 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 78 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 67.5 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 56 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 87 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 60.9 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 32 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 51 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 43.5 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 34 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 57 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 42.0 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 27 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 87 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 66.1 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 50 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 94 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 81.9 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 66 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 77 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 56.5 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 43 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 56 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 46.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 36 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 54 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 47.8 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 40 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 89 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 69.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 53 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 90 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 65.0 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 44 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 52 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 42.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 34 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 56 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 41.9 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 33 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 51 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 41.1 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 34 </td><!-- --></tr><!-- --></table></td><td _ngcontent-app-root-c189="" class="ng-star-inserted"><table _ngcontent-app-root-c189="" aria-labelledby="Days data"><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> Max </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> Avg </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> Min </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 22 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 12.9 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 3 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 12 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 8.7 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 15 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 9.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 5 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 10 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 6.2 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 12 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 5.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 12 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 8.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 7 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 20 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 11.5 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 3 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 20 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 11.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 10 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 5.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 14 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 9.5 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 5 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 16 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 11.8 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 7 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 12 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 8.2 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 3 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 12 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 6.7 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 3 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 23 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 13.2 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 6 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 12 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 6.8 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 10 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 6.8 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 3 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 18 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 9.0 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 5 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 18 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 12.5 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 23 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 10.9 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 5 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 18 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 11.0 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 7 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 22 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 15.6 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 7 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 28 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 12.0 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 23 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 14.2 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 5 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 25 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 13.9 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 8 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 14 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 6.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 21 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 10.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 3 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 20 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 14.6 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 6 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 24 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 18.8 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 13 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 17 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 11.7 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 8 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 15 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 10.1 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 5 </td><!-- --></tr><!-- --></table></td><td _ngcontent-app-root-c189="" class="ng-star-inserted"><table _ngcontent-app-root-c189="" aria-labelledby="Days data"><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> Max </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> Avg </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> Min </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.0 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.9 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.5 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.3 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.5 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.3 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.2 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.1 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.1 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.0 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.9 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.2 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.1 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.9 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.9 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.8 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.7 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.1 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.0 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.9 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.0 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.9 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.8 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.1 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.1 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.0 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.2 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.5 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.4 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.1 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.1 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.3 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.2 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.1 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.1 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.0 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.8 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.9 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.7 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.7 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.1 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.9 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.9 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.5 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.1 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.5 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.0 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.9 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.7 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.6 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.0 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.8 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.6 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.1 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.0 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.4 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.3 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.3 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.1 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.5 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.8 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.7 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.5 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.0 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.9 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 29.8 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.1 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.1 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.0 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.1 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.1 </td><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 30.1 </td><!-- --></tr><!-- --></table></td><td _ngcontent-app-root-c189="" class="ng-star-inserted"><table _ngcontent-app-root-c189="" aria-labelledby="Days data"><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> Total </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.05 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 2.39 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.33 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><tr _ngcontent-app-root-c189="" class="ng-star-inserted"><td _ngcontent-app-root-c189="" class="ng-star-inserted"> 0.00 </td><!-- --></tr><!-- --></table></td><!-- --></tr></tbody></table>
"""

save_monthly_weather('2023','11',text)
    
